In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_gaussian_quantiles


In [ ]:
# Construct dataset
X1, y1 = make_gaussian_quantiles(cov=2.,
                                 n_samples=200, n_features=2,
                                 n_classes=2, random_state=1)
X2, y2 = make_gaussian_quantiles(mean=(3, 3), cov=1.5,
                                 n_samples=300, n_features=2,
                                 n_classes=2, random_state=1)
X = np.concatenate((X1, X2))
y = np.concatenate((y1, - y2 + 1))

In [ ]:
# Uniform then select
X = np.random.uniform(-5.0, 5.0, (500,2))
y = np.zeros((X.shape[0],))
for i in range(X.shape[0]):
    y[i] = 1 if (2.0< X[i,0]< 3.5 and 2.0<X[i,1]<4.2) or (-2.5< X[i,0]< 0.0 and -0.5<X[i,1]<1.0) else 0

In [ ]:
# Balanced set
X1=np.array([np.random.uniform(2.0, 3.5, (50,)), np.random.uniform(2.0, 4.2, (50,))]).T
y1=np.ones((50,))
X2=np.array([np.random.uniform(-2.5, 0.0, (100,)), np.random.uniform(-0.5, 1.0, (100,))]).T
y2=np.ones((100,))
X3=list()
while len(X3) < 400:
    tmp = np.random.uniform(-5.0, 5.0, (2,))
    if (2.0< tmp[0]< 3.5 and 2.0<tmp[1]<4.2) or (-2.5< tmp[0]< 0.0 and -0.5<tmp[1]<1.0):
        continue
    X3.append(tmp)
X3 = np.array(X3)
y3 = np.zeros((400,))
X = np.concatenate((X1, X2, X3))
y = np.concatenate((y1, y2, y3))


In [ ]:
# Create and fit an AdaBoosted decision tree
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),
                         algorithm="SAMME.R",
                         n_estimators=200)
bdt.fit(X, y)


In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
bdt = GaussianProcessClassifier()
bdt.fit(X,y)

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
bdt = GaussianProcessRegressor()
bdt.fit(X,y)

In [ ]:
plot_colors = "br"
plot_step = 0.02
class_names = "AB"

fig = plt.figure(figsize=(10, 5))

# Plot the decision boundaries
plt.subplot(121)
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                     np.arange(y_min, y_max, plot_step))

if type(bdt) is not GaussianProcessRegressor:
    Z = bdt.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, cmap=plt.cm.Paired)
else:
    Z = bdt.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    cs = plt.contour(xx, yy, Z)
plt.axis("tight")

# Plot the training points
for i, n, c in zip(range(2), class_names, plot_colors):
    idx = np.where(y == i)
    plt.scatter(X[idx, 0], X[idx, 1],
                c=c, cmap=plt.cm.Paired,
                s=20, edgecolor='k',
                label="Class %s" % n)
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.legend(loc='upper right')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Decision Boundary')

In [ ]:
fig.savefig("AdaBoost_DT_D1.png", bbox_inches="tight")